In [1]:
# !pip uninstall httpx httpcore
# !pip install --upgrade httpx httpcore

In [ ]:
import pandas as pd
import os
from openai import OpenAI
import math
import json
import re

In [1]:
with open('config.json') as config_file:
    config = json.load(config_file)
    open_api_key = config['open_api_key']
language = 'French'
client = OpenAI(api_key=open_api_key)

In [54]:
def get_sets_to_do(pre_directory, post_directory):
    os.makedirs(os.path.join(language, 'Vocabulary', post_directory), exist_ok = True)
    donesets = os.listdir(os.path.join(language, 'Vocabulary', post_directory))
    print(donesets)
    if '.DS_Store' in donesets:
        donesets.remove('.DS_Store')
    allsets = os.listdir(os.path.join(language, 'Vocabulary', pre_directory))
    notdonesets = [i for i in allsets if not i in donesets]
    todosets = notdonesets#['900-1000.csv']
    if '.DS_Store' in donesets:
        todosets.remove('.DS_Store')
    print(todosets)
    return todosets

todosets = get_sets_to_do('Split_Sets','ChatGPT_Sets')

['100-200.csv', '0-100.csv']
['800-900.csv', '600-700.csv', '700-800.csv', '300-400.csv', '900-1000.csv', '400-500.csv', '500-600.csv', '200-300.csv']


Clean

In [55]:
def clean(language = language):
    df = pd.read_csv(language +'/Vocabulary/French1000POS.csv', sep = ',')
#     df = df[[language, 'POS', 'English']]
#     df.to_csv(language +'/Vocabulary/Clean')
    return df
df = clean()
df

,Rank,French,English,POS
0,1,être,to be; being,"verb, auxiliary verb, noun"
1,2,avoir,to have,"verb, auxiliary verb"
2,3,je,I,personal pronoun
3,4,de,"of, from, by, than, in, with (see #28)",preposition
4,5,ne,not,adverb
...,...,...,...,...
995,996,j,NaN,noun
996,997,espoir,NaN,noun
997,998,tueur,NaN,noun
998,999,grandir,NaN,verb


VOCABULARY SPLIT

In [56]:
def split_into_studysets(df, language, splitsize = 100):
    os.makedirs(os.path.join(language, 'Vocabulary', 'Split_Sets'), exist_ok = True)
    frequencydf = df
    # frequencydf['Frequency'] = frequencydf['Frequency'].astype(int)
    # frequencydf['cumsum'] = frequencydf['Frequency'].cumsum()/(0.01 * frequencydf['Frequency'].sum())
    columns = [language, 'POS']
    if 'English' in df.columns:
        columns.append('English')
    for i in range(math.floor(len(frequencydf)/splitsize)):
        subdf = frequencydf[splitsize*i: splitsize*(i+1)][columns]
        subdf.to_csv(language +'/Vocabulary/Split_Sets/' + str(splitsize*i) + '-' + str(splitsize*(i+1)) + '.csv')
    print('remainder=', -(len(frequencydf)%splitsize))
    remainder = -(len(frequencydf)%splitsize)
    if remainder > 0:
        remaindersubdf = frequencydf[remainder:][columns]
        if len(remaindersubdf) != splitsize:
            remaindersubdf.to_csv(language + '/Vocabulary/Split_Sets/' + 'Remainder.csv')

    return subdf
df = split_into_studysets(df, language = language)

remainder= 0


CHATGPT

In [57]:
todosets = get_sets_to_do('Split_Sets','ChatGPT_Sets')

['100-200.csv', '0-100.csv']
['800-900.csv', '600-700.csv', '700-800.csv', '300-400.csv', '900-1000.csv', '400-500.csv', '500-600.csv', '200-300.csv']


In [58]:
def generate_sentence_in_target_language(variable_word, POS, words_to_include, model = 'gpt-4o-2024-05-13', tenses = ['present', 'future', 'past']):
    """
    Generate a sentence in Russian using a specific word and part of speech, with words from a limited vocabulary, along with its English translation.

    Parameters:
    variable_word (str): The word to include in the sentence.
    POS (str): The part of speech the word should operate as.
    words_to_include (tuple): A list of words to try to include, and how many
    model (str): The model to use for generating the sentence (default is 'gpt-4o-2024-05-13').
    
    # vocabulary (list): A list of words to use in the sentence.
    # tenses (list)

    Returns:
    str: A sentence in Russian and its English translation, separated by a newline.
    """
    
    print(variable_word)
    words_to_include = list(words_to_include[0].sample(words_to_include[1], replace=False))
    # vocabulary = vocabulary
    # vocabulary_str = ', '.join(vocabulary)
    prompt = (
              # f"Use only the following Russian words: {vocabulary_str}. "
        
              f"Create a simple sentence in f{language} using basic vocabulary and containing the word '{variable_word}' "
              f"operating as a {POS} part of speech. Also, provide the English translation of the sentence separated by a newline."
              f"Try to include the following words in the sentence: {words_to_include}"
              # f"Use only the following tenses: {tenses}. "
              )

    try:
        # Make the API request
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=50,
            temperature=0.2,
        )

        # Extract and return the output
        output_message = response.choices[0].message.content.strip()
        return output_message

    except Exception as e:
        return f"An error occurred: {str(e)}"

    # Extract and return the output
    output_message = response.choices[0].message.content.strip()
    return output_message
# R1000 = pd.read_csv('Russian/Vocabulary/Clean')

In [59]:
# ', '.join(R1000.iloc[:100].Russian)

In [60]:
# for index, row in R1000.iloc[:3].iterrows():
#     print(row.Russian, row.POS)
#     
#     sentence = generate_sentence_in_russian(row.Russian, row.POS, (R1000.iloc[300:310].Russian, 2))
#     print(sentence)

In [61]:
def chatonset(s_set_df, set_name, n = 2):
    words_to_include = (s_set_df[language], n)
    s_set_df['ChatGPT_Sentence'] = s_set_df.apply(lambda row: generate_sentence_in_target_language(row[language], row['POS'], words_to_include), axis = 1)
    s_set_df.to_csv(os.path.join(language, 'Vocabulary','ChatGPT_Sets', set_name))
    return s_set_df

In [63]:
def do_all_sets(pre_directory, post_directory, func, language = language):
    os.makedirs(os.path.join(language, 'Vocabulary', post_directory), exist_ok = True)
    allsets = os.listdir(os.path.join(language, 'Vocabulary', pre_directory))
    # todosets = get_sets_to_do(pre_directory, post_directory)
    todosets = ['0-100.csv', '100-200.csv']
    # print(allsets)
    # print(todosets)
    for s_set in todosets:
        if s_set[0] != '.':
        # if s_set == '500-600.csv':
            # print(s_set)
            s_set_df = pd.read_csv(
                os.path.join(language, 'Vocabulary', pre_directory, s_set)
            )
            # print(s_set_df)
            func(s_set_df, s_set)
            
    return

In [64]:
do_all_sets('Split_Sets', 'ChatGPT_Sets', chatonset)

être
avoir
je
de
ne
pas
le
la
tu
vous
il
et
à
un
l'
qui
aller
les
en
ça
faire
tout
on
que
ce
une
mes
d'
pour
se
des
dire
pouvoir
vouloir
mais
me
nous
dans
elle
savoir
du
où
y
t'
bien
voir
plus
non
te
mon
au
avec
moi
si
quoi
devoir
oui
ils
comme
s'
venir
sur
toi
ici
rien
lui
bon
là
suivre
pourquoi
parler
prendre
cette
quand
alors
une chose
par
son
croire
aimer
falloir
comment
très
ou
passer
penser
aussi
jamais
attendre
trouver
laisser
petit
merci
même
sa
ta
autre
arriver
ces
donner
regarder
encore
appeler
est-ce que
peu
homme
partir
ma
toujours
jour
femme
temps
maintenant
notre
vie
deux
mettre
rester
sans
seul
arrêter
vraiment
connaître
quelque
sûr
tuer
mourir
demander
juste
peut-être
dieu
fois
oh
père
comprendre
sortir
personne
an
trop
chez
fille
aux
monde
ami
vrai
après
mal
besoin
accord
ses
avant
monsieur
enfant
grand
entendre
voilà
chercher
heure
mieux
tes
aider
mère
déjà
beau
essayer
quel
vos
depuis
quelqu'un
beaucoup
revenir
donc
plaire
maison
gens
nuit
ah
soir
nom
bonjour
jouer
l

ADD TRANSLATION INFORMATION:

In [65]:
from deep_translator import GoogleTranslator

In [66]:
language[:2].lower()

'fr'

In [47]:
def add_set_translation_information(s_set_df, set_name):
    # Apply the translation
    s_set_df['Translation'] = s_set_df.apply(
        lambda row: GoogleTranslator(source= language[:2].lower(), target='en').translate(row[language]), axis=1
    )

    # Save to CSV
    s_set_df.to_csv(os.path.join(language, 'Vocabulary/Translated Sets', set_name), index=False)
    return s_set_df

In [48]:
translation_todosets = get_sets_to_do('ChatGPT_Sets', 'Translated Sets')

['0-100.csv']
['100-200.csv']


In [49]:
for set in translation_todosets:
    print(set)
    do_all_sets('ChatGPT_Sets', 'Translated Sets', add_set_translation_information)

100-200.csv


Make into Quizlet

In [96]:
def make_quizlet_set(df, set_name):
    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].apply(lambda x: re.sub(r'\n+', '\n', x))
    
    # Removing text within parentheses that start with 'see'
    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.replace(r'\(see[^)]*\)', '', regex=True)
    
    # Removing any extra spaces that might result from the replacement
    df['ChatGPT_Sentence'] = df['ChatGPT_Sentence'].str.strip()

 
    # Create the 'quizlet' column with POS : Translation and exactly two newlines between Translation and ChatGPT_Sentence
    df['quizlet'] = df['POS'] + ' : ' + '*' + df['English'] + '*' + '\n\n' + df['ChatGPT_Sentence']


    df = df[[language, 'quizlet']]
    output_path = os.path.join(language, 'Vocabulary/Quizlet Sets', set_name)
    df.to_csv(output_path, index=False, quoting=csv.QUOTE_MINIMAL)
    # print(f"CSV file saved to {output_path}")
    return df

In [97]:
import re
import csv
quizlet_todosets = get_sets_to_do('ChatGPT_Sets', 'Quizlet Sets')
for set in quizlet_todosets:
    do_all_sets('ChatGPT_Sets', 'Quizlet Sets', make_quizlet_set)
    
def makequizlet_step2(input_file):
    output_file = input_file 
    with open(input_file, 'r', newline='', encoding='utf-8') as infile:
        reader = csv.reader(infile)
        rows = list(reader)

    # Write the output CSV file with extra newlines
    with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)
        for row in rows:
            writer.writerow(row)
            writer.writerow([])  # Add a blank row
            writer.writerow([])  # Add a blank row
            writer.writerow([])

    print(f"Modified CSV file saved as {output_file}")
    
for file in os.listdir(os.path.join(language, 'Vocabulary/Quizlet Sets')):
    if file[0] != '.':
        print(file)
        makequizlet_step2(os.path.join(language, 'Vocabulary/Quizlet Sets/' + file))

['.DS_Store']
['.DS_Store', '100-200.csv', '0-100.csv']
100-200.csv
Modified CSV file saved as French/Vocabulary/Quizlet Sets/100-200.csv
0-100.csv
Modified CSV file saved as French/Vocabulary/Quizlet Sets/0-100.csv


In [89]:
# def remove_quotes_from_csv(file_path):
#     # Read the CSV file and remove quotes
#     with open(file_path, 'r', newline='') as infile:
#         reader = csv.reader(infile)
#         rows = [[cell.replace('"', '') for cell in row] for row in reader]
#     
#     # Write the cleaned data back to the CSV file
#     with open(file_path, 'w', newline='') as outfile:
#         writer = csv.writer(outfile)
#         writer.writerows(rows)

In [90]:
# for file in os.listdir('Russian/Vocabulary/Quizlet Sets'):
#     if file[0] != '.':
#         print(file)
#         makequizlet_step2('Russian/Vocabulary/Quizlet Sets/' + file)

800-900.csv
Modified CSV file saved as Russian/Vocabulary/Quizlet Sets/800-900.csv
100-200.csv
Modified CSV file saved as Russian/Vocabulary/Quizlet Sets/100-200.csv
600-700.csv
Modified CSV file saved as Russian/Vocabulary/Quizlet Sets/600-700.csv
700-800.csv
Modified CSV file saved as Russian/Vocabulary/Quizlet Sets/700-800.csv
500-600.csv
Modified CSV file saved as Russian/Vocabulary/Quizlet Sets/500-600.csv
